
### Lab | Handling Data Imbalance in Classification Models


#### 1.Import the required libraries and modules that you would need.

In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

#### 2.Read that data into Python and call the dataframe donors.

In [2]:
numerical = pd.read_csv('numerical7_02.csv')
categorical = pd.read_csv('categorical7_02.csv')
target = pd.read_csv('target7_02.csv')

In [3]:
# Check if everything have same rows in order to merge
print(numerical.shape)
print(categorical.shape)
print(target.shape)

(95412, 323)
(95412, 13)
(95412, 3)


In [4]:
numerical.head()
numerical = numerical.drop(columns = ["Unnamed: 0"])

In [5]:
numerical.head()

,ODATEDW,TCODE,DOB,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,...,LASTGIFT,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
0,8901,0,3712,60.000000,5.0,NaN,0,0,39,34,...,10.0,9512,8911,9003.0,4.0,7.741935,95515,0,4,39.0
1,9401,1,5202,46.000000,6.0,9.0,16,0,15,55,...,25.0,9512,9310,9504.0,18.0,15.666667,148535,0,2,1.0
2,9001,1,0,61.611649,3.0,1.0,2,0,20,29,...,5.0,9512,9001,9101.0,12.0,7.481481,15078,1,4,60.0
3,8701,0,2801,70.000000,1.0,4.0,2,0,23,14,...,10.0,9512,8702,8711.0,9.0,6.812500,172556,1,4,41.0
4,8601,0,2001,78.000000,3.0,2.0,60,1,28,9,...,15.0,9601,7903,8005.0,14.0,6.864865,7112,1,2,26.0


In [6]:
categorical.head()
categorical = categorical.drop(columns = ["Unnamed: 0"])

In [7]:
categorical.head()

,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,SOLIH,VETERANS,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B
0,IL,36,U,F,0,NaN,NaN,L,E,C,T,2
1,CA,14,H,M,3,NaN,NaN,L,G,A,S,1
2,NC,43,U,M,3,NaN,NaN,L,E,C,R,2
3,CA,44,U,F,3,NaN,NaN,L,E,C,R,2
4,FL,16,H,F,3,12.0,NaN,L,F,A,S,2


In [8]:
target.head()
target = target.drop(columns = ["Unnamed: 0"])

In [9]:
target.head()

,TARGET_B,TARGET_D
0,0,0.0
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0


In [10]:
data = pd.concat((numerical,categorical,target),axis=1) # concat lines up the index. Index 0 with index 0 of each datasets. Rest index before is needed
data.head()

,ODATEDW,TCODE,DOB,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,...,DATASRCE,SOLIH,VETERANS,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,TARGET_B,TARGET_D
0,8901,0,3712,60.000000,5.0,NaN,0,0,39,34,...,0,NaN,NaN,L,E,C,T,2,0,0.0
1,9401,1,5202,46.000000,6.0,9.0,16,0,15,55,...,3,NaN,NaN,L,G,A,S,1,0,0.0
2,9001,1,0,61.611649,3.0,1.0,2,0,20,29,...,3,NaN,NaN,L,E,C,R,2,0,0.0
3,8701,0,2801,70.000000,1.0,4.0,2,0,23,14,...,3,NaN,NaN,L,E,C,R,2,0,0.0
4,8601,0,2001,78.000000,3.0,2.0,60,1,28,9,...,3,12.0,NaN,L,F,A,S,2,0,0.0


In [11]:
donors = pd.DataFrame(data, columns = data.columns)
donors.head()


,ODATEDW,TCODE,DOB,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,...,DATASRCE,SOLIH,VETERANS,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,TARGET_B,TARGET_D
0,8901,0,3712,60.000000,5.0,NaN,0,0,39,34,...,0,NaN,NaN,L,E,C,T,2,0,0.0
1,9401,1,5202,46.000000,6.0,9.0,16,0,15,55,...,3,NaN,NaN,L,G,A,S,1,0,0.0
2,9001,1,0,61.611649,3.0,1.0,2,0,20,29,...,3,NaN,NaN,L,E,C,R,2,0,0.0
3,8701,0,2801,70.000000,1.0,4.0,2,0,23,14,...,3,NaN,NaN,L,E,C,R,2,0,0.0
4,8601,0,2001,78.000000,3.0,2.0,60,1,28,9,...,3,12.0,NaN,L,F,A,S,2,0,0.0


In [12]:
cols =[]
for colname in donors.columns:
    cols.append(colname.lower().replace(' ','_'))
donors.columns = cols

donors.head()

,odatedw,tcode,dob,age,income,wealth1,hit,malemili,malevet,vietvets,...,datasrce,solih,veterans,rfa_2r,rfa_2a,geocode2,domain_a,domain_b,target_b,target_d
0,8901,0,3712,60.000000,5.0,NaN,0,0,39,34,...,0,NaN,NaN,L,E,C,T,2,0,0.0
1,9401,1,5202,46.000000,6.0,9.0,16,0,15,55,...,3,NaN,NaN,L,G,A,S,1,0,0.0
2,9001,1,0,61.611649,3.0,1.0,2,0,20,29,...,3,NaN,NaN,L,E,C,R,2,0,0.0
3,8701,0,2801,70.000000,1.0,4.0,2,0,23,14,...,3,NaN,NaN,L,E,C,R,2,0,0.0
4,8601,0,2001,78.000000,3.0,2.0,60,1,28,9,...,3,12.0,NaN,L,F,A,S,2,0,0.0


#### 3. Check the datatypes of all the columns in the data.

In [13]:
donors.dtypes

odatedw       int64
tcode         int64
dob           int64
age         float64
income      float64
             ...   
geocode2     object
domain_a     object
domain_b      int64
target_b      int64
target_d    float64
Length: 336, dtype: object

#### 4. Check for null values in the dataframe. Replace the null values using the methods learned in class.

In [14]:
nulls_percent_df = pd.DataFrame(donors.isna().sum()*100/len(data)).reset_index()
nulls_percent_df


,index,0
0,odatedw,0.0
1,tcode,0.0
2,dob,0.0
3,age,0.0
4,income,0.0
...,...,...
331,geocode2,0.0
332,domain_a,0.0
333,domain_b,0.0
334,target_b,0.0


In [15]:
nulls_percent_df.columns = ['column_name', 'nulls_percentage']
nulls_percent_df

,column_name,nulls_percentage
0,odatedw,0.0
1,tcode,0.0
2,dob,0.0
3,age,0.0
4,income,0.0
...,...,...
331,geocode2,0.0
332,domain_a,0.0
333,domain_b,0.0
334,target_b,0.0


In [16]:
nulls_percent_df[nulls_percent_df['nulls_percentage']!=0]

,column_name,nulls_percentage
5,wealth1,46.882992
135,msa,0.138347
136,adi,0.138347
137,dma,0.138347
315,nextdate,10.452564
316,timelag,10.452564
327,solih,93.501866
328,veterans,89.072653


In [17]:
# Since, there are a few of columns we are going to consider the threshhold above 0. 
columns_above_threshold = nulls_percent_df[nulls_percent_df['nulls_percentage']>0]

In [18]:
drop_columns_list = list(columns_above_threshold['column_name'])
print(drop_columns_list)

['wealth1', 'msa', 'adi', 'dma', 'nextdate', 'timelag', 'solih', 'veterans']


###### 4.1 Dealing with Wealth1 NaN

In [19]:
donors['wealth1'].value_counts(dropna=False)

NaN    44732
9.0     7585
8.0     6793
7.0     6198
6.0     5825
5.0     5280
4.0     4810
3.0     4237
2.0     4085
1.0     3454
0.0     2413
Name: wealth1, dtype: int64

In [20]:
# we can see that the behaviour is mostly categorical. We will chante type to object and assing nan to median

In [21]:
donors['wealth1'] = donors['wealth1'].astype('object')
donors['wealth1'] = donors['wealth1'].fillna('5.0') 

##### 4.2 Dealing with MSA, ADI and DMA

In [22]:
# Dealth with them in first lab of the week. Taking the same aproach (132 NaN for each feature)

In [23]:
donors['adi'].value_counts()

13.0     7296
51.0     4622
65.0     3765
57.0     2836
105.0    2617
         ... 
651.0       1
103.0       1
601.0       1
161.0       1
147.0       1
Name: adi, Length: 204, dtype: int64

In [24]:
donors['adi'] = donors['adi'].fillna('13.0')

In [25]:
donors['msa'].value_counts()

0.0       21333
4480.0     4606
1600.0     4059
2160.0     2586
520.0      1685
          ...  
9140.0        1
3200.0        1
9280.0        1
743.0         1
8480.0        1
Name: msa, Length: 298, dtype: int64

In [26]:
donors['msa'] = donors['msa'].fillna('0.0')

In [27]:
donors['dma'].value_counts()

803.0    7296
602.0    4632
807.0    3765
505.0    2839
819.0    2588
         ... 
569.0       1
554.0       1
584.0       1
552.0       1
516.0       1
Name: dma, Length: 206, dtype: int64

In [28]:
donors['msa'] = donors['msa'].fillna('803.0')

##### 4.3 Dealing with nextdate

In [29]:
donors['nextdate'].value_counts(dropna=False)
# Again, consult with management. We don´t consider relevant to build the model. So we are going to drop it

NaN       9973
9504.0    2253
9412.0    1970
8703.0    1959
9512.0    1870
          ... 
8107.0       1
7408.0       1
8207.0       1
8104.0       1
8412.0       1
Name: nextdate, Length: 189, dtype: int64

##### 4.4 Dealing with timelag

In [30]:
donors['timelag'].value_counts(dropna=False) 

NaN       9973
5.0       8584
4.0       8490
3.0       8485
6.0       6648
          ... 
89.0         1
1044.0       1
53.0         1
442.0        1
1088.0       1
Name: timelag, Length: 69, dtype: int64

In [31]:
donors['timelag'] = donors['timelag'].fillna(np.ceil(np.median(donors['timelag'])))

In [32]:
donors['timelag'].value_counts()

5.0       8584
4.0       8490
3.0       8485
6.0       6648
2.0       6275
          ... 
89.0         1
77.0         1
61.0         1
58.0         1
1088.0       1
Name: timelag, Length: 68, dtype: int64

##### 4.5 Dealing with solih

In [33]:
donors['solih'].value_counts(dropna=False) # Solih is the represents the number of times we can solicit donations by year   
# We would need to discuss the manager if worth it getting data about this. From now we will drop the column since we can´t get the information

NaN     89212
12.0     5693
0.0       296
1.0        94
2.0        75
3.0        19
4.0        16
6.0         7
Name: solih, dtype: int64

##### 4.6 Dealing with veterans

In [34]:
# Again, discuss with manager. For the moment we are dropping it 

In [35]:
donors['veterans'].value_counts(dropna=False)

NaN    84986
Y      10426
Name: veterans, dtype: int64

In [36]:
# To sum up from the previous list we are going to drop 'nextdate', solih and veterans

In [37]:
 drop_columns_list

['wealth1', 'msa', 'adi', 'dma', 'nextdate', 'timelag', 'solih', 'veterans']

In [38]:
donors = donors.drop(columns=['nextdate','solih','veterans'], axis=1)

In [39]:
donors = donors.dropna()

In [46]:
donors.shape

(85319, 333)

#### 5. Split the data into numerical and catagorical. Decide if any columns need their dtype changed.



In [71]:
numericals=(donors.drop(columns=['target_b','target_d'])).select_dtypes(include=np.number)
categoricals=donors.select_dtypes(include=object)
targetb = donors['target_b']

##### 5.1 Clean and select features from numerical

In [72]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
numerical_scaled=scaler.fit_transform(numericals)

In [73]:
pd.DataFrame(numerical_scaled).head()

,0,1,2,3,4,5,6,7,8,9,...,311,312,313,314,315,316,317,318,319,320
0,0.426523,0.000000,0.382286,0.608247,0.666667,0.000000,0.000000,0.393939,0.343434,0.181818,...,0.927939,0.003676,0.006816,0.498047,0.0,1.000000,0.622951,0.673077,0.0,0.333333
1,0.784946,0.000014,0.535736,0.463918,0.833333,0.066390,0.000000,0.151515,0.555556,0.111111,...,0.969489,0.016544,0.015182,0.774514,0.0,0.333333,0.000000,0.250000,1.0,0.000000
2,0.498208,0.000014,0.000000,0.624862,0.333333,0.008299,0.000000,0.202020,0.292929,0.333333,...,0.937311,0.011029,0.006541,0.078617,1.0,1.000000,0.967213,0.807692,1.0,0.333333
3,0.283154,0.000000,0.288465,0.711340,0.000000,0.008299,0.000000,0.232323,0.141414,0.313131,...,0.906175,0.008272,0.005835,0.899769,1.0,1.000000,0.655738,0.826923,1.0,0.333333
4,0.211470,0.000000,0.206076,0.793814,0.333333,0.248963,0.010101,0.282828,0.090909,0.535354,...,0.822972,0.012868,0.005890,0.037080,1.0,0.333333,0.409836,0.288462,1.0,0.333333


In [74]:
X = numerical_scaled
y = targetb

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
kbest = SelectKBest(chi2, k=15).fit_transform(X,y)
selected=pd.DataFrame(kbest)
selected.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.010101,0.079833,0.105833,0.000000,0.020202,0.070707,0.131313,0.272727,0.000000,0.020202,0.050505,0.341463,0.045226,1.000000,0.333333
1,0.000000,0.911333,0.869667,0.000000,0.979798,0.989899,0.989899,0.989899,0.949495,0.919192,0.919192,0.024390,0.045226,0.333333,0.000000
2,0.020202,0.082833,0.091000,0.000000,0.000000,0.010101,0.060606,0.181818,0.000000,0.000000,0.020202,0.341463,0.045226,1.000000,0.333333
3,0.000000,0.166667,0.210500,0.000000,0.101010,0.252525,0.505051,0.696970,0.101010,0.010101,0.080808,0.170732,0.045226,1.000000,0.333333
4,0.989899,0.096000,0.099000,0.381818,0.000000,0.010101,0.020202,0.161616,0.000000,0.252525,0.585859,0.195122,0.492462,0.333333,0.333333


In [78]:
kbest = SelectKBest(chi2, k=20).fit(X,y)
df=pd.DataFrame(data=kbest.scores_,columns=['score'])
df['column'] = numericals.columns
df.sort_values(by=['score'], ascending=False).head(15)


,score,column
316,157.036542,rfa_2f
310,51.222164,lastdate
111,23.667293,hvp1
112,22.178849,hvp2
113,18.217589,hvp3
304,18.150932,cardgift
116,18.043424,hvp6
114,12.744965,hvp4
23,10.553603,eth2
130,9.867662,rp1


In [81]:
columns=df.sort_values(by=['score'], ascending=False).head(15)['column']
list(columns)

['rfa_2f',
 'lastdate',
 'hvp1',
 'hvp2',
 'hvp3',
 'cardgift',
 'hvp6',
 'hvp4',
 'eth2',
 'rp1',
 'rp2',
 'hv1',
 'hv2',
 'ethc4',
 'domain_b']

In [86]:
numerical_def = numericals[['rfa_2f',
 'lastdate',
 'hvp1',
 'hvp2',
 'hvp3',
 'cardgift',
 'hvp6',
 'hvp4',
 'eth2',
 'rp1',
 'rp2',
 'hv1',
 'hv2',
 'ethc4',
 'domain_b']]

In [91]:
donors = pd.concat((categoricals, numerical_def,target), axis=1)
donors.head()

,wealth1,msa,adi,state,homeownr,gender,rfa_2r,rfa_2a,geocode2,domain_a,...,hvp4,eth2,rp1,rp2,hv1,hv2,ethc4,domain_b,TARGET_B,TARGET_D
0,5.0,0.0,177.0,IL,U,F,L,E,C,T,...,27.0,1.0,2.0,5.0,479.0,635.0,0.0,2.0,0,0.0
1,9.0,4480.0,13.0,CA,H,M,L,G,A,S,...,98.0,0.0,91.0,91.0,5468.0,5218.0,0.0,1.0,0,0.0
2,1.0,0.0,281.0,NC,U,M,L,E,C,R,...,18.0,2.0,0.0,2.0,497.0,546.0,0.0,2.0,0,0.0
3,4.0,9340.0,67.0,CA,U,F,L,E,C,R,...,69.0,0.0,1.0,8.0,1000.0,1263.0,0.0,2.0,0,0.0
4,2.0,5000.0,127.0,FL,H,F,L,F,A,S,...,16.0,98.0,25.0,58.0,576.0,594.0,21.0,2.0,0,0.0


#### 6. Concatenate numerical and categorical back together again for your X dataframe. Designate the Target as y.

#### 7. Split the data into a training set and a test set.

In [ ]:
# Before spliting we are going to do the feature selection on numericals

##### 7.1 Feature selection on numericals

In [ ]:
# First, we need to scale our numericals

In [ ]:
# to look at VarianceThresholds we need all the variable to be on the same scale
# we will use MinMaxScaler for this
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
numerical_scaled = scaler.fit_transform(numericals)

In [ ]:
pd.DataFrame(numerical_scaled).head()

In [ ]:
from sklearn.feature_selection import VarianceThreshold 
var_threshold = 0.02
sel = VarianceThreshold(threshold=(var_threshold))

# 1) default is threshold = 0 :eliminate all coumns that are identical for all rows
# 2) in practice we would scale the columns first, and then apply threshold, or apply 
#    a different threshold for different columns

# This drops the columns that have a variance less than this threshold
sel = sel.fit(numerical_scaled)
temp = sel.transform(numerical_scaled)
temp = pd.DataFrame(temp)
print(numerical_scaled.shape)
print(temp.shape)

In [ ]:
temp.head()

In [ ]:


Split the data into a training set and a test set.
Split further into train_num and train_cat. Also test_num and test_cat.
Scale the features either by using normalizer or a standard scaler. (train_num, test_num)
Encode the categorical features using One-Hot Encoding or Ordinal Encoding. (train_cat, test_cat)
fit only on train data transform both train and test
again re-concatenate train_num and train_cat as X_train as well as test_num and test_cat as X_test
Fit a logistic regression model on the training data.
Check the accuracy on the test data.
Note: So far we have not balanced the data.

Managing imbalance in the dataset

Check for the imbalance.
Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.
Each time fit the model and see how the accuracy of the model has changed.